In [ ]:
import emblaze
from emblaze.utils import Field, ProjectionTechnique
import numpy as np
import pandas as pd

In [ ]:
def load_bnhp():
    df = pd.read_csv("housing.csv", header=None, sep='\s+')
    df.columns = ["crim", "zn", "indus", "chas", "nox", "rm", "age", "dis", "rad", "tax", "ptratio", "black", "lstat", "medv"]

    all_values = df[df["medv"] < 50].values
    means = np.mean(all_values, axis=0)
    stds = np.std(all_values, axis=0)
    means[df.columns.values.tolist().index("chas")] = 0
    stds[df.columns.values.tolist().index("chas")] = 1

    normalized = (all_values - means) / stds

    x_train = normalized[:,:-1]
    y_train = normalized[:,-1]
    
    return x_train, y_train > 0.0

def load_mnist(size=2000):
    """Loads only the test data"""
    from tensorflow.keras.datasets import mnist
    _, (x_test, y_test) = mnist.load_data()
    indexes = np.random.choice(x_test.shape[0], size=size, replace=False)
    return x_test[indexes].reshape(-1, 28*28), y_test[indexes]

In [ ]:
# Load data

X, Y = load_mnist()

In [ ]:
# Generate the projections

emb = emblaze.Embedding({Field.POSITION: X, Field.COLOR: Y})
emb.compute_neighbors(metric='jaccard')

variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.UMAP) for _ in range(5)
])

thumbnails = emblaze.ImageThumbnails(X.reshape(-1, 28, 28, 1))

In [ ]:
# Display the widget

w = emblaze.Viewer(embeddings=variants, thumbnails=thumbnails)
w

## Selection

We can read and write to the current selection in the interface.

In [ ]:
w.selectedIDs

In [ ]:
w.selectedIDs = list(range(100, 105)) # Choose ID numbers to select

## Alignment

We can also choose points to align all visualizations to. Try selecting a group of points by Cmd/Ctrl+click and dragging, then clicking Align. Then when you move to a different frame in the UI above, the points you selected will attempt to stay constant.

The below code can be used to align to a group of points that you select programmatically. The `align_to` function implemented below automatically adds the set of nearest neighbors to the alignment set, which can lead to a better alignment.

In [ ]:
def align_to(ids, include_vicinity=False):
    if not ids:
        w.align_to_points(None, [])
    else:
        ids = list(ids)
        vicinity = set()
        for neighbors in w.frames[0].mat("highlight", ids=ids):
            vicinity |= set(neighbors.tolist())
        w.align_to_points(ids, list(vicinity))

align_to(w.selectedIDs, include_vicinity=True)

In [ ]:
# Reset alignment
align_to(None)